# @realDonaldTrump

## Import necessary libraries

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pymongo
import numpy as np
import pandas as pd
from datetime import datetime


url = "https://twitter.com/search?f=tweets&vertical=default&q=lang%3Aen%20from%3ArealDonaldTrump%20since%3A2015-06-1%20until%3A2016-05-21&src=typd"


## Opens webdriver and url
driver = webdriver.Firefox()
driver.get(url)
driver.maximize_window()

## Scroll down to bottom of a Twitter page
lastHeight = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        break
    lastHeight = newHeight

pho = BeautifulSoup(driver.page_source, "html.parser")


## Connection to Mongo DB
try:
    conn = pymongo.MongoClient()
    print "Connected successfully..."
except pymongo.errors.ConnectionFailure, e:
    print "Could not connect to MongoDB: %s" % e
conn

mydb = conn['twits']

## Extract information from url

dat = pd.DataFrame({'time': [], 'twit': [], 'tag': [], 'at': [], \
                    're': [], 'like': [], 'loc': []})

for t in pho.find_all('div', {'class': 'content'}):
    time = t.find('a', {'class': 'tweet-timestamp'})['title'].encode('UTF-8')
    try:
        twit = t.find('p', {'class': 'TweetTextSize'}).text.encode('UTF-8')
    except:
        pass
    tag = []
    for l in t.find_all('a', {'class': 'twitter-hashtag'}):
        tag.append(l.text.encode('UTF-8')[1:])
    if tag == []:
        tag = np.NaN
    at = []
    for a in t.find_all('a', {'class': 'twitter-atreply'}):
        try:
            at.append(a['href'].encode('UTF-8')[1:])
        except:
            pass
    if at == []:
        at = np.NaN
    thumbs = t.find_all('span', \
                        {'class': 'ProfileTweet-actionCountForPresentation'})
    try:
        re = thumbs[0].text.encode('UTF-8')
        if re[-1] == 'K':
            re = int(float(re[:-1].encode('UTF-8')) * 1000)
        else:
            re = int(float(re.encode('UTF-8')))
    except:
        pass
    try:
        like = thumbs[2].text.encode('UTF-8')
        if like[-1] == 'K':
            like = int(float(like[:-1].encode('UTF-8'))*1000)
        else:
            like = int(float(like.encode('UTF-8')))
    except:
        pass
    if t.find_all('span', {'class': 'Tweet-geo'}) == []:
        loc = np.NaN
    else:
        loc = t.find('span', {'class': 'Tweet-geo'})['title'].encode('UTF-8')
    entry = {'time': time, 'twit': twit, 'tag': [tag], 'at': [at], \
             're': re, 'like': like, 'loc': loc}
    dat = dat.append([pd.DataFrame(entry)], ignore_index=True)
    mydb.trump.insert(entry)

## Export as a csv for safekeeping
dat.to_csv('trumptwit.csv', encoding='utf-8')

print "Mission Complete..."
driver.close()
print "Have a nice day!"

In [55]:
twits = pd.read_csv('trumptwit.csv', index_col='Unnamed: 0')

# Processing data

twits['time'] = pd.to_datetime(twits['time'], format='%I:%M %p - %d %b %Y')

In [56]:
twits

,at,like,loc,re,tag,time,twit
0,[montgomeriefdn],2800.0,NaN,1000.0,NaN,2016-05-20 16:36:00,"@montgomeriefdn Colin, great to have you at T..."
1,"[montgomeriefdn, TrumpTurnberry]",7500.0,NaN,1800.0,NaN,2016-05-20 16:27:00,"""@montgomeriefdn: @TrumpTurnberry Best Links C..."
2,[TGowdySC],13000.0,NaN,4600.0,[Trump2016],2016-05-20 15:50:00,Thank you for your wonderful endorsement today...
3,[NRA],14000.0,NaN,4400.0,"[MakeAmericaGreatAgain, Trump2016]",2016-05-20 14:55:00,"Great day in Kentucky with Wayne LaPierre, Chr..."
4,"[NRA, GOPconvention]",16000.0,NaN,6600.0,"[Trump2016, 2A, NRA, NRAAM]",2016-05-20 13:06:00,Thank you! An honor to be the first candidate ...
5,[nytimes],12000.0,United States,3600.0,NaN,2016-05-20 09:11:00,Failing @NYTimes will always take a good story...
6,[piersmorgan],15000.0,United States,4700.0,NaN,2016-05-20 09:09:00,"Thanks @piersmorgan! \n""Trump is the most unpr..."
7,[NRA],11000.0,NaN,6300.0,"[SecondAmendment, 2A]",2016-05-20 08:36:00,Crooked Hillary Clinton- discussing the #Secon...
8,NaN,7600.0,"New York, NY",2700.0,[NRAAM],2016-05-20 08:15:00,I am on my way! See you all soon!https://twitt...
9,[JasonDovEsq],6000.0,NaN,2000.0,NaN,2016-05-20 07:30:00,"""A Call for Unity"" by Jason Greenblatt @JasonD..."


In [90]:
print twits.count()
twits.describe()

at      3659
like    5889
loc      642
re      5889
tag     1344
time    5889
twit    5889
dtype: int64


,like,re
count,5889.000000,5889.000000
mean,5231.653761,2021.047886
std,6139.156165,2538.561256
min,14.000000,2.000000
25%,1400.000000,626.000000
50%,2900.000000,1200.000000
75%,6900.000000,2600.000000
max,110000.000000,85000.000000


at      3659
like    5889
loc      642
re      5889
tag     1344
time    5889
twit    5889
dtype: int64